In [8]:
import requests
import json
from dotenv import load_dotenv
import os


load_dotenv()

def fetch_line_reports():
    # URL API
    url = 'https://api.navitia.io/v1/coverage/fr-idf/line_reports'

    # clé api from env
    api_key = os.getenv('token_navitia')
    headers = {
        'Authorization': api_key
    }
    
    # requete
    response = requests.get(url, headers=headers)

  
    if response.status_code == 200:
       
        data = response.json()
        # ecriture requete positif en hson
        with open('line_reports.json', 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print("Data written to line_reports.json")
    else:
        
        print('Failed to retrieve data: ', response.status_code)

# Appel func
fetch_traffic_reports()


Data written to line_reports.json


In [18]:
import pandas as pd
from datetime import datetime

with open('line_reports.json') as f:
    data = json.load(f)

disruptions_data = []

for disruption in data['disruptions']:
    for impacted_object in disruption['impacted_objects']:
        pt_object = impacted_object['pt_object']
        # Check 'stop_point' key existe 
        
        if 'stop_point' in pt_object:  
            stop_point = pt_object['stop_point']
            location_name = stop_point['name']
            location_label = stop_point['label']
            longitude = stop_point['coord']['lon']
            latitude = stop_point['coord']['lat']
            address_label = stop_point['address']['label']
            fare_zone = stop_point['fare_zone']['name']
            transport_type = pt_object['embedded_type']
        else:
            # none si on trouve pas
            location_name = location_label = longitude = latitude = address_label = fare_zone = transport_type = None
        
        # check pour le mode de transport quand il se trouve dedans
        if 'line' in pt_object and 'physical_modes' in pt_object['line']:
            physical_mode = pt_object['line']['physical_modes'][0]['name']
        else:
            physical_mode = None

        disruption_info = {
            "disruption_id": disruption['id'],
            "status": disruption['status'],
            "cause": disruption['cause'],
            "category": disruption['category'],
            "severity_name": disruption['severity']['name'],
            "severity_effect": disruption['severity']['effect'],
            "severity_color": disruption['severity']['color'],
            "severity_priority": disruption['severity']['priority'],
            "message_text": disruption['messages'][0]['text'],
            "application_period_begin": datetime.strptime(disruption['application_periods'][0]['begin'], '%Y%m%dT%H%M%S'),
            "application_period_end": datetime.strptime(disruption['application_periods'][0]['end'], '%Y%m%dT%H%M%S'),
            "location_name": location_name,
            "location_label": location_label,
            "longitude": longitude,
            "latitude": latitude,
            "address_label": address_label,
            "fare_zone": fare_zone,
            "transport_type": transport_type,
            "physical_mode": physical_mode
        }
        
        disruptions_data.append(disruption_info)

# dataframe gang
df_disruptions = pd.DataFrame(disruptions_data)

df_disruptions

,disruption_id,status,cause,category,severity_name,severity_effect,severity_color,severity_priority,message_text,application_period_begin,application_period_end,location_name,location_label,longitude,latitude,address_label,fare_zone,transport_type,physical_mode
0,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,Clément Ader,Clément Ader (Argenteuil),2.24347,48.942583,Rue de la Tour Billy (Argenteuil),4,stop_point,None
1,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,Charles de Gaulle - Henri Barbusse,Charles de Gaulle - Henri Barbusse (Argenteuil),2.245179,48.940498,Avenue du Général de Gaulle (Argenteuil),4,stop_point,None
2,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,Place du 11 Novembre,Place du 11 Novembre (Argenteuil),2.239094,48.936521,Nouvelle Impasse Duguay (Argenteuil),4,stop_point,None
3,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,Avenue du Château,Avenue du Château (Argenteuil),2.233204,48.933282,Allée Simone Veil (Argenteuil),4,stop_point,None
4,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,None,None,None,None,None,None,None,Bus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,f7588214-069a-11ef-98d5-0a58a9feac02,active,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,"<p><span style=""background-color:rgb(255,255,2...",2024-04-29 17:15:00,2024-04-30 23:59:00,None,None,None,None,None,None,None,Bus
100,f7588214-069a-11ef-98d5-0a58a9feac02,active,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,"<p><span style=""background-color:rgb(255,255,2...",2024-04-29 17:15:00,2024-04-30 23:59:00,None,None,None,None,None,None,None,Bus
101,f7588214-069a-11ef-98d5-0a58a9feac02,active,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,"<p><span style=""background-color:rgb(255,255,2...",2024-04-29 17:15:00,2024-04-30 23:59:00,None,None,None,None,None,None,None,Bus
102,f7588214-069a-11ef-98d5-0a58a9feac02,active,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,"<p><span style=""background-color:rgb(255,255,2...",2024-04-29 17:15:00,2024-04-30 23:59:00,None,None,None,None,None,None,None,Bus


In [1]:
from meteofrance_api import MeteoFranceClient

# Create a client instance
client = MeteoFranceClient()

# Search for a location
places = client.search_places('Paris')

# Get the forecast for the first result
forecast = client.get_forecast_for_place(places[0])

# Print the forecast data
print(forecast.daily_forecast)


[{'dt': 1714435200, 'T': {'min': 12.6, 'max': 19.1, 'sea': None}, 'humidity': {'min': 65, 'max': 90}, 'precipitation': {'24h': 0.8}, 'uv': 4, 'weather12H': {'icon': 'p3j', 'desc': 'Très nuageux'}, 'sun': {'rise': 1714451455, 'set': 1714503928}}, {'dt': 1714521600, 'T': {'min': 14.1, 'max': 20.2, 'sea': None}, 'humidity': {'min': 65, 'max': 90}, 'precipitation': {'24h': 8.5}, 'uv': 4, 'weather12H': {'icon': 'p13j', 'desc': 'Pluies éparses'}, 'sun': {'rise': 1714537753, 'set': 1714590415}}, {'dt': 1714608000, 'T': {'min': 12.1, 'max': 14.1, 'sea': None}, 'humidity': {'min': 80, 'max': 95}, 'precipitation': {'24h': 5.5}, 'uv': 3, 'weather12H': {'icon': 'p13j', 'desc': 'Pluies éparses'}, 'sun': {'rise': 1714624052, 'set': 1714676902}}, {'dt': 1714694400, 'T': {'min': 10.5, 'max': 16.6, 'sea': None}, 'humidity': {'min': 65, 'max': 95}, 'precipitation': {'24h': 0}, 'uv': 4, 'weather12H': {'icon': 'p2j', 'desc': 'Eclaircies'}, 'sun': {'rise': 1714710353, 'set': 1714763389}}, {'dt': 1714780800

In [4]:
rain_data = client.get_rain()
print(rain_data.forecast)


TypeError: MeteoFranceClient.get_rain() missing 2 required positional arguments: 'latitude' and 'longitude'

In [6]:
from meteofrance_api import MeteoFranceClient
from datetime import datetime, timedelta

client = MeteoFranceClient()

# Supposons que cette fonction puisse récupérer des données historiques
def fetch_weather_data(place, start_date, end_date):
    current_date = start_date
    weather_data = []
    while current_date <= end_date:
        forecast = client.get_historical_data(place, current_date)
        weather_data.append(forecast)
        current_date += timedelta(days=1)  # Incrément journalier
    return weather_data

# Recherche du lieu
places = client.search_places('Paris')

# Définir les dates de début et de fin pour l'année 2023
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# Récupérer les données météo
weather_data = fetch_weather_data(places[0], start_date, end_date)
print(weather_data)


AttributeError: 'MeteoFranceClient' object has no attribute 'get_historical_data'

In [8]:
import pandas as pd

df = pd.read_csv('Q_01_previous-1950-2022_RR-T-Vent.csv', delimiter=';')
df

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJ,RR,QRR,TN,QTN,...,DXI2,QDXI2,HXI2,QHXI2,FXI3S,QFXI3S,DXI3S,QDXI3S,HXI3S,QHXI3S
0,1010001,ANGLEFORT,45.900000,5.766667,280,19500101,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1010001,ANGLEFORT,45.900000,5.766667,280,19500102,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1010001,ANGLEFORT,45.900000,5.766667,280,19500103,13.8,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1010001,ANGLEFORT,45.900000,5.766667,280,19500104,5.4,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1010001,ANGLEFORT,45.900000,5.766667,280,19500105,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955878,1457001,VONNAS,46.218333,4.990000,190,19920327,0.9,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
955879,1457001,VONNAS,46.218333,4.990000,190,19920328,0.7,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
955880,1457001,VONNAS,46.218333,4.990000,190,19920329,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
955881,1457001,VONNAS,46.218333,4.990000,190,19920330,18.6,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import pandas as pd

# Function to parse descriptive files
def parse_descriptive_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    field_names = {}
    for line in lines:
        parts = line.strip().split(":")
        if len(parts) == 2:
            field, description = parts
            field_names[field.strip()] = description.strip()
    return field_names

# Parse descriptive files for both datasets
desc_fields_autres_params = parse_descriptive_file('Q_descriptif_champs_autres-parametres.csv')
desc_fields_rr_t_vent = parse_descriptive_file('Q_descriptif_champs_RR-T-Vent.csv')

# Load datasets
dataset_autres_params = pd.read_csv('Q_01_1852-1949_autres-parametres.csv', sep=';')
dataset_rr_t_vent = pd.read_csv('Q_01_1852-1949_RR-T-Vent.csv', sep=';')

# Rename columns using the parsed descriptive fields
dataset_autres_params.rename(columns=desc_fields_autres_params, inplace=True)
dataset_rr_t_vent.rename(columns=desc_fields_rr_t_vent, inplace=True)

# Display the first few rows of each dataset with new column names
print("Autres Paramètres Dataset:")
print(dataset_autres_params.head())
print("\nRR-T-Vent Dataset:")
print(dataset_rr_t_vent.head())


FileNotFoundError: [Errno 2] No such file or directory: 'Q_descriptif_champs_RR-T-Vent(1).csv'